In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os

from tensorflow.contrib.tensorboard.plugins import projector


LOG_DIR =os.path.join(os.getcwd(),'fmrisample')
NAME_TO_VISUALISE_VARIABLE = "fmri_embedding"


path_for_mnist_sprites =  os.path.join(LOG_DIR,'fmris.png')
path_for_mnist_metadata =  os.path.join(LOG_DIR,'metadata.tsv')

In [4]:
class Scan(object):
    def __init__(self, activations, timestamp, step, word=None, prev_word=None, next_word=None):
        self.activations = activations
        self.timestamp = timestamp
        self.word = word
        self.prev_word = prev_word
        self.next_word = next_word
        self.step = step


        
def read_and_prepare_data():
    scan_objects = np.load("../data/subject_1_scan_objects.npy")
    embeddings = np.load("../data/lstm_emb_objects.npy")
    # print(len(scan_objects.item().get(1)))
    # print(embeddings.item().get(1))
    block_id = 1
    brain_scans = []
    brain_scan_steps = []
    current_word = []
    lstm_embeddings = []
    words = []
    for scan_obj in scan_objects.item().get(block_id):
        # print(scan_obj.step, scan_obj.word, scan_obj.timestamp)
        brain_scans.append(scan_obj.activations[0])
        brain_scan_steps.append(scan_obj.step)
        current_word.append(scan_obj.word)
        lstm_embeddings.append(embeddings.item().get(block_id)[scan_obj.step])
        if scan_obj.prev_word == None:
            scan_obj.prev_word =""
        if scan_obj.next_word == None:
            scan_obj.next_word =""
        words.append(scan_obj.prev_word+"_"+scan_obj.word+"_"+scan_obj.next_word)

    brain_scans = np.asarray(brain_scans)
    brain_scan_steps = np.asarray(brain_scan_steps)
    current_word = np.asarray(current_word)
    lstm_embeddings = np.asarray(lstm_embeddings)
    words = np.asarray(words)

    min_voxel_value = np.min(brain_scans)
    max_voxel_value = np.max(brain_scans)
    print("brain scans min max: %f %f" % (min_voxel_value, max_voxel_value))
    normalized_brain_scans = (brain_scans - min_voxel_value) / (max_voxel_value - min_voxel_value)
    nmin_voxel_value = np.min(normalized_brain_scans)
    nmax_voxel_value = np.max(normalized_brain_scans)
    print("normalized brain scans min max: %f %f" % (nmin_voxel_value, nmax_voxel_value))
    print(len(normalized_brain_scans))
    return lstm_embeddings, normalized_brain_scans, words


In [5]:
lstm_embeddings, normalized_brain_scans, words = read_and_prepare_data()

#lstm_embeddings, normalized_brain_scans, words = lstm_embeddings[:100], normalized_brain_scans[:100], words[:100]
embedding_var = tf.Variable(lstm_embeddings, name=NAME_TO_VISUALISE_VARIABLE)
summary_writer = tf.summary.FileWriter(LOG_DIR)

brain scans min max: -91.000000 1068.000000
normalized brain scans min max: 0.000000 1.000000
326


In [6]:
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = embedding_var.name

# Specify where you find the metadata
embedding.metadata_path = path_for_mnist_metadata #'metadata.tsv'

# Specify where you find the sprite (we will create this later)
#embedding.sprite.image_path = path_for_mnist_sprites #'mnistdigits.png'
#embedding.sprite.single_image_dim.extend([28,28])

# Say that you want to visualise the embeddings
projector.visualize_embeddings(summary_writer, config)

In [7]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()
saver.save(sess, os.path.join(LOG_DIR, "model.ckpt"), 1)

'/Users/samiraabnar/Codes/SequentialNeuroSemantics/data/fmrisample/model.ckpt-1'

In [8]:
def create_sprite_image(images):
    """Returns a sprite image consisting of images passed as argument. Images should be count x width x height"""
    if isinstance(images, list):
        images = np.array(images)
    img_h = images.shape[1]
    img_w = images.shape[2]
    n_plots = int(np.ceil(np.sqrt(images.shape[0])))
    
    
    spriteimage = np.ones((img_h * n_plots ,img_w * n_plots ))
    
    for i in range(n_plots):
        for j in range(n_plots):
            this_filter = i * n_plots + j
            if this_filter < images.shape[0]:
                this_img = images[this_filter]
                spriteimage[i * img_h:(i + 1) * img_h,
                  j * img_w:(j + 1) * img_w] = this_img
    
    return spriteimage

def vector_to_matrix_mnist(mnist_digits):
    """Reshapes normal mnist digit (batch,28*28) to matrix (batch,28,28)"""
    return np.reshape(mnist_digits,(-1,28,28))

def invert_grayscale(mnist_digits):
    """ Makes black white, and white black """
    return 1-mnist_digits

In [9]:
#to_visualise = batch_xs
#to_visualise = vector_to_matrix_mnist(to_visualise)
#to_visualise = invert_grayscale(to_visualise)

#sprite_image = create_sprite_image(to_visualise)

#plt.imsave(path_for_mnist_sprites,sprite_image,cmap='gray')
#plt.imshow(sprite_image,cmap='gray')

In [10]:
with open(path_for_mnist_metadata,'w') as f:
    f.write("Index\tLabel\tifTarget\n")
    for index,label in enumerate(words):
        f.write("%d\t%s\t%d\n" % (index,label,if_target[index-1]))